Langchain Document - https://docs.langchain.com/oss/python/integrations/retrievers

In [16]:
from dotenv import load_dotenv
load_dotenv()
from langchain_huggingface import HuggingFaceEmbeddings
# create embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

### **Wikipedia Retriever**

In [17]:
from langchain_community.retrievers import WikipediaRetriever

# Initialize the retriever (optional: set language and top_k)
retriever = WikipediaRetriever(top_k_results=3, lang='en')

In [18]:
query = "The political stand of India in Russia-Ukraine war"

docs = retriever.invoke(query)

In [19]:
for i, doc in enumerate(docs):
    print(f"\n---result{i+1} ---")
    print(f"content:\n{doc.page_content}")


---result1 ---
content:
The Russo-Ukrainian war began in February 2014 and is ongoing. Following Ukraine's Revolution of Dignity, Russia occupied Crimea and annexed it from Ukraine. It then supported Russian separatist armed groups who started a war in the eastern Donbas region against Ukraine's military. In 2018, Ukraine declared the region to be occupied by Russia. The first eight years of conflict also involved naval incidents and cyberwarfare. In February 2022, Russia launched a full-scale invasion of Ukraine and began occupying more of the country, starting the current phase of the war, the biggest conflict in Europe since World War II. The war has resulted in a refugee crisis and hundreds of thousands of deaths.
In early 2014, the Euromaidan protests led to the Revolution of Dignity and the ousting of Ukraine's pro-Russian president Viktor Yanukovych. Shortly after, pro-Russian protests began in parts of southeastern Ukraine, while unmarked Russian troops occupied Crimea. Russia

### **Vector Store Retriever**


In [13]:
from langchain_chroma import Chroma
from langchain_core.documents import Document

# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
    Document(page_content="Now you have to import Chroma form langchain_chroma module")
]

# Vector store
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="MyCollection"
)

In [14]:
# vector store retriever
retriever = vector_store.as_retriever(search_kwargs={'k':2})

In [15]:
query = "Tell me about chroma"
result = retriever.invoke(query)

for i, doc in enumerate(result):
    print(f"\n---result{i+1}---")
    print(doc.page_content)


---result1---
Now you have to import Chroma form langchain_chroma module

---result2---
Now you have to import Chroma form langchain_chroma module


### **MMR - Maximal Marginal Relevence**

In [20]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

vector_store = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [37]:
# MMR retrival
mmr_retriever = vector_store.as_retriever(
    search_type='mmr',
    lambda_mult = 0.5, # Diversity of results returned by MMR
    search_kwargs = {'k': 3}
)

In [38]:
query = "Tell me about Langchain"

results = mmr_retriever.invoke(query)

for i, doc in enumerate(results):
    print(f"\n---result{i+1}---")
    print(doc.page_content)


---result1---
LangChain supports Chroma, FAISS, Pinecone, and more.

---result2---
LangChain is used to build LLM based applications.

---result3---
Embeddings are vector representations of text.


### **Multiquery Retriever**

In [77]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

hf_llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-9b-it",
    task="text-generation"
)
model = ChatHuggingFace(llm=hf_llm)

In [78]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

vector_store = FAISS.from_documents(
    documents=all_docs,
    embedding=embedding_model
)

In [79]:
# Different Retrievers
similarity_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [80]:
from langchain_classic.retrievers import MultiQueryRetriever

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever= vector_store.as_retriever(search_type='mmr', lambda_mult=0.5, search_kwargs={"k":6}),
    llm = model
)

In [81]:
# Query
query = "How to improve my health?"

# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [ ]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

### **Contextual Compression Retriever**

In [83]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [84]:
vector_store = FAISS.from_documents(documents=docs, embedding= embedding_model)

In [85]:
base_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [86]:
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm=model)

In [88]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [89]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [90]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.

--- Result 3 ---
Photosynthesis does not occur in animal cells.
